# 프로젝트: 더 멋진 번역기 만들기

| 평가문항 | 상세기준 |
| --- | --- |
| 1. 번역기 모델 학습에 필요한 텍스트 데이터 전처리가 잘 이루어졌다. | 데이터 정제, SentencePiece를 활용한 토큰화 및 데이터셋 구축의 과정이 지시대로 진행되었다. |
| 2. Transformer 번역기 모델이 정상적으로 구동된다. | Transformer 모델의 학습과 추론 과정이 정상적으로 진행되어, 한-영 번역기능이 정상 동작한다. |
| 3. 테스트 결과 의미가 통하는 수준의 번역문이 생성되었다. | 제시된 문장에 대한 그럴듯한 영어 번역문이 생성되며, 시각화된 Attention Map으로 결과를 뒷받침한다. |

# 1. 필요한 라이브러리 설치 및 데이터 다운로드
아래 링크에서 korean-english-park.train.tar.gz 를 사용할 예정입니다. 다운로드할 필요는 없습니다.

- jungyeul/korean-parallel-corpora

☁️클라우드 환경에서는 위 데이터를 미리 준비해 놓았으니 연결만 시켜줍시다. 우측 하단의 Cloud shell을 열어주세요.
아래와 같이 공유 디렉토리에 저장된 데이터를 가리키는 심볼릭 링크를 생성해 주시면 됩니다.

```
$ ln -s ~/data ~/aiffel/transformer/data
```

In [ ]:
# 나눔 글꼴 install
! sudo apt -qq -y install fonts-nanum

In [ ]:
# SentencePiece 설치

! pip install sentencepiece

In [3]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm

%config InlineBackend.figure_format = 'retina'
 
fontpath = '/usr/share/fonts/truetype/nanum/NanumBarunGothic.ttf'
font = fm.FontProperties(fname=fontpath, size=9)
plt.rc('font', family='NanumBarunGothic') 
mpl.font_manager._rebuild()

print("한글 테스트")

한글 테스트


# 2. 데이터 정제 및 토큰화
1. set 데이터형이 중복을 허용하지 않는다는 것을 활용해 중복된 데이터를 제거하도록 합니다. 데이터의 병렬 쌍이 흐트러지지 않게 주의하세요! 중복을 제거한 데이터를 cleaned_corpus 에 저장합니다.


2. 정제 함수를 아래 조건을 만족하게 정의하세요.
- 모든 입력을 소문자로 변환합니다.
- 알파벳, 문장부호, 한글만 남기고 모두 제거합니다.
- 문장부호 양옆에 공백을 추가합니다.
- 문장 앞뒤의 불필요한 공백을 제거합니다.

3. 한글 말뭉치 kor_corpus 와 영문 말뭉치 eng_corpus 를 각각 분리한 후, 정제하여 토큰화를 진행합니다! 토큰화에는 Sentencepiece를 활용하세요. 첨부된 공식 사이트를 참고해 아래 조건을 만족하는 generate_tokenizer() 함수를 정의합니다. 최종적으로 ko_tokenizer 과 en_tokenizer 를 얻으세요. en_tokenizer에는 set_encode_extra_options("bos:eos") 함수를 실행해 타겟 입력이 문장의 시작 토큰과 끝 토큰을 포함할 수 있게 합니다.

- [google/sentencepiece](https://github.com/google/sentencepiece)

> 단어 사전을 매개변수로 받아 원하는 크기의 사전을 정의할 수 있게 합니다. (기본: 20,000)

>학습 후 저장된 model 파일을 SentencePieceProcessor() 클래스에 Load()한 후 반환합니다.

>특수 토큰의 인덱스를 아래와 동일하게 지정합니다.
PAD : 0 / BOS : 1 / EOS : 2 / UNK : 3

4. 토크나이저를 활용해 토큰의 길이가 50 이하인 데이터를 선별하여 src_corpus 와 tgt_corpus 를 각각 구축하고, 텐서 enc_train 과 dec_train 으로 변환하세요! (❗모든 데이터를 사용할 경우 학습에 굉장히 오랜 시간이 걸립니다.)

In [4]:
import tensorflow as tf
import numpy as np

from sklearn.model_selection import train_test_split
import sentencepiece as spm
import matplotlib.ticker as ticker
import matplotlib.pyplot as plt

import time
import re
import os
import io

from tqdm import tqdm
from tqdm import tqdm_notebook
import random

data_dir = '/content/drive/MyDrive/Aiffel/GD8/data'
kor_path = data_dir + "/korean-english-park.train.ko"
eng_path = data_dir + "/korean-english-park.train.en"

print(tf.__version__)

2.8.0


In [5]:
def clean_corpus(kor_path, eng_path):
    with open(kor_path, "r") as f: kor = f.read().splitlines()
    with open(eng_path, "r") as f: eng = f.read().splitlines()
    assert len(kor) == len(eng)
    
    cleaned_corpus = list(set(zip(kor, eng)))
    
    return cleaned_corpus
    
cleaned_corpus = clean_corpus(kor_path, eng_path)

In [6]:
print(cleaned_corpus[0:3])

[('전문가의 회의 노 대통령, 주한미군 동북아 분쟁 개입 반대\xa0 \xa0 \xa0', "Roh clarified that Korea won't allow USFK to be involved in any dispute in Northeast Asia."), ('이시장의 보좌관은 이 600만원은 총사용시간에 대한 요금 중 이시장이 지불해야하는 요금으로 충분하다고 주장하고 있으나, 체육진흥회에서 공개한 영수증을 보면 이 요금은 단지 2005년 7월에서 12월까지의 사용요금일 뿐이다.', 'His staff claims that that amount covered his part of the fees for the entire time, but the receipt issued by the association says it paid for only July through December 2005.'), ('맨유의 골키퍼인 에드윈 판데르 사르는 후반전에 결정적인 세이브를 기록하기도 했지만 그렇게 심한 압박은 받지 않았다.', 'goalkeeper Edwin van der Sar made a string of saves, especially in the second half, but was not often seriously tested.')]


In [7]:
print('정제된 corpus 데이터의 수는', len(cleaned_corpus), '개 입니다.')

정제된 corpus 데이터의 수는 78968 개 입니다.


In [8]:
def preprocess_sentence(sentence):
    # 모든 입력은 소문자로 변환
    sentence = sentence.lower()
    # 알파벳, 문장부호, 한글만 남기고 모두 제거
    sentence = re.sub(r"[^a-zA-Zㄱ-ㅎ가-힣ㅏ-ㅣ.,?!]+", " ", sentence)
    # 문장부호 양 사이드에 공백 추가
    sentence = re.sub(r"([,.?!])", r" \1 ", sentence)
    sentence = re.sub(r'[" "]+', " ", sentence)
    
    return sentence

In [9]:
# Sentencepiece를 활용하여 학습한 tokenizer를 생성합니다.
def generate_tokenizer(corpus, model_type='unigram', vocab_size=32000, lang="ko"): 
    
    input_file = f'{lang}_spm_input.txt'

    with open(input_file, 'w', encoding='utf-8') as f:
        for sent in corpus:
            f.write('{}\n'.format(sent))
        
    sp_model_root='sentencepiece'
    if not os.path.isdir(sp_model_root): os.mkdir(sp_model_root)
    
    prefix = 'tokenizer_%s_%s' % (lang,model_type+str(vocab_size))
    prefix = os.path.join(sp_model_root, prefix) # 저장될 tokenizer 모델에 붙는 이름
    pad_id=0
    bos_id=1 #<start> token을 1으로 설정
    eos_id=2 #<end> token을 2으로 설정
    unk_id=3 #<unknown> token을 3으로 설정
    character_coverage = 1.0 # to reduce character set 
    # model_type = model_type # Choose from unigram (default), bpe, char, or word
    input_argument = '--input=%s --pad_id=%s --bos_id=%s --eos_id=%s --unk_id=%s --model_prefix=%s --vocab_size=%s --character_coverage=%s --model_type=%s'
    cmd = input_argument%(input_file, pad_id, bos_id, eos_id, unk_id, prefix, vocab_size, character_coverage, model_type)
                      
    spm.SentencePieceTrainer.Train(cmd)
    tokenizer = spm.SentencePieceProcessor()
    tokenizer.Load(f'{prefix}.model')
    
    return tokenizer

In [10]:
cleaned_corpus[1]

('이시장의 보좌관은 이 600만원은 총사용시간에 대한 요금 중 이시장이 지불해야하는 요금으로 충분하다고 주장하고 있으나, 체육진흥회에서 공개한 영수증을 보면 이 요금은 단지 2005년 7월에서 12월까지의 사용요금일 뿐이다.',
 'His staff claims that that amount covered his part of the fees for the entire time, but the receipt issued by the association says it paid for only July through December 2005.')

In [11]:
SRC_VOCAB_SIZE = TGT_VOCAB_SIZE = 20000

eng_corpus = []
kor_corpus = []

for pair in cleaned_corpus:
    k, e = pair[0],pair[1]

    kor_corpus.append(preprocess_sentence(k))
    eng_corpus.append(preprocess_sentence(e))

ko_tokenizer = generate_tokenizer(kor_corpus, 'unigram', SRC_VOCAB_SIZE, "ko")
en_tokenizer = generate_tokenizer(eng_corpus, 'unigram', TGT_VOCAB_SIZE, "en")
en_tokenizer.set_encode_extra_options("bos:eos")

True

In [12]:
cleaned_corpus[0]

('전문가의 회의 노 대통령, 주한미군 동북아 분쟁 개입 반대\xa0 \xa0 \xa0',
 "Roh clarified that Korea won't allow USFK to be involved in any dispute in Northeast Asia.")

In [13]:
cleaned_corpus[100]

('남북간의 최고 대화 채널인 장관급회담은 북한 수도, 평양에서 2월 27일부터 3월 2일까지 열릴 것이라고 북한의 개성에서 회담을 가진 남북 실무대표들이 전했다.',
 'The Cabinet-level talks the highest dialogue channel between the two Koreas will be held in the North Korean capital, Pyongyang, from February 27 through to March 2, according to a joint statement adopted at a lower-level meeting in the North Korean border city of Kaesong.')

In [14]:
ko_test_txts = ['한편 이번 금융사고는 개인이 저지른 것 중 규모가 가장 클 뿐만아니라 유럽에서 발생한 금융사기 중 최대 규모이다.']
en_test_txts = ['Polanski failed to show up for sentencing and never returned to the United States.']


In [15]:
def testSP(sp, txts):
    tokens_list = []
    for txt in txts:
        tokens = sp.encode_as_pieces(txt)
        #ids = sp.encode_as_ids(txt)
        tokens_list.append(tokens)
    return tokens_list

In [16]:
testSP(ko_tokenizer, ko_test_txts)

[['▁한편',
  '▁이번',
  '▁금융사고',
  '는',
  '▁개인',
  '이',
  '▁저지른',
  '▁것',
  '▁중',
  '▁규모',
  '가',
  '▁가장',
  '▁클',
  '▁뿐만',
  '아니라',
  '▁유럽에서',
  '▁발생한',
  '▁금융',
  '사',
  '기',
  '▁중',
  '▁최대',
  '▁규모',
  '이다',
  '.']]

In [17]:
testSP(en_tokenizer, en_test_txts)

[['<s>',
  '▁',
  'P',
  'ol',
  'anski',
  '▁fail',
  'ed',
  '▁to',
  '▁show',
  '▁up',
  '▁for',
  '▁sentenc',
  'ing',
  '▁and',
  '▁never',
  '▁return',
  'ed',
  '▁to',
  '▁the',
  '▁',
  'U',
  'nit',
  'ed',
  '▁',
  'S',
  't',
  'ates',
  '.',
  '</s>']]

In [18]:
from tqdm import tqdm_notebook # Process 과정을 보기 위해 tqdm_notebook이 사용됩니다.
import tqdm
import tensorflow as tf

src_corpus = []
tgt_corpus = []

assert len(kor_corpus) == len(eng_corpus)

# 토큰의 길이가 50 이하인 문장만 남깁니다.
for idx in tqdm_notebook(range(len(kor_corpus))):
    src = ko_tokenizer.EncodeAsIds(kor_corpus[idx])
    tgt = en_tokenizer.EncodeAsIds(eng_corpus[idx])
    
    if len(src) <= 50 and len(tgt) <= 50:
        src_corpus.append(src)
        tgt_corpus.append(tgt)
        
# 패딩처리를 완료하여 학습용 데이터를 완성합니다. 
enc_train = tf.keras.preprocessing.sequence.pad_sequences(src_corpus, padding='post')
dec_train = tf.keras.preprocessing.sequence.pad_sequences(tgt_corpus, padding='post')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  # This is added back by InteractiveShellApp.init_path()


  0%|          | 0/78968 [00:00<?, ?it/s]

# 3. 모델설계

In [19]:
def positional_encoding(pos, d_model):
    def cal_angle(position, i):
        return position / np.power(10000, int(i)/d_model)
    
    def get_posi_angle_vec(position):
        return [cal_angle(position, i) for i in range(d_model)]
    
    sinusoid_table = np.array([get_posi_angle_vec(pos_i) for pos_i in range(pos)])
    
    sinusoid_table[:, 0::2] = np.sin(sinusoid_table[:, 0::2])
    sinusoid_table[:, 1::2] = np.cos(sinusoid_table[:, 1::2])
    
    return sinusoid_table

In [20]:
class MultiHeadAttention(tf.keras.layers.Layer):
    def __init__(self, d_model, num_heads):
        super(MultiHeadAttention, self).__init__()
        self.num_heads = num_heads
        self.d_model = d_model
        
        self.depth = d_model // self.num_heads
        
        self.W_q = tf.keras.layers.Dense(d_model)  
        self.W_k = tf.keras.layers.Dense(d_model)
        self.W_v = tf.keras.layers.Dense(d_model)
        
        self.linear = tf.keras.layers.Dense(d_model)
    
    def scaled_dot_product_attention(self, Q, K, V, mask):
        d_k = tf.cast(K.shape[-1], tf.float32)
        
        QK = tf.matmul(Q, K, transpose_b=True)
        scaled_qk = QK / tf.math.sqrt(d_k)
        
        if mask is not None:
            scaled_qk += (mask * -1e9)
        
        attentions = tf.nn.softmax(scaled_qk, axis=-1)
        out = tf.matmul(attentions, V)
        return out, attentions 
    
    def scaled_dot_product_attention(self, Q, K, V, mask):
        d_k = tf.cast(K.shape[-1], tf.float32)
        
        # Scaled QK 값 구하기
        QK = tf.matmul(Q, K, transpose_b=True)
        scaled_qk = QK / tf.math.sqrt(d_k)
        
        if mask is not None:
            scaled_qk += (mask * -1e9)
        
        # 1. Attention Weights 값 구하기 -> attentions
        attentions = tf.nn.softmax(scaled_qk, axis=-1)
        # 2. Attention 값을 V에 곱하기 -> out
        out = tf.matmul(attentions, V)
        return out, attentions
    
    def split_heads(self, x):
        """
        Embedding된 입력을 head의 수로 분할하는 함수
        
        x: [ batch x length x emb ]
        return: [ batch x length x heads x self.depth ]
        """
        bsz = x.shape[0]
        split_x = tf.reshape(x, (bsz, -1, self.num_heads, self.depth))
        split_x = tf.transpose(split_x, perm=[0, 2, 1, 3])
        return split_x
    
    def combine_heads(self, x):
        """
        분할된 Embedding을 하나로 결합하는 함수
        
        x: [ batch x length x heads x self.depth ]
        return: [ batch x length x emb ]
        """
        bsz = x.shape[0]
        combined_x = tf.transpose(x, perm=[0, 2, 1, 3])
        combined_x = tf.reshape(combined_x, (bsz, -1, self.d_model))
        return combined_x
    
    def call(self, Q, K, V, mask):
        """
        Step 1: Linear_in(Q, K, V) -> WQ, WK, WV
        Step 2: Split Heads(WQ, WK, WV) -> WQ_split, WK_split, WV_split
        Step 3: Scaled Dot Product Attention(WQ_split, WK_split, WV_split)
                 -> out, attention_weights
        Step 4: Combine Heads(out) -> out
        Step 5: Linear_out(out) -> out
        """
        WQ = self.W_q(Q)
        WK = self.W_k(K)
        WV = self.W_v(V)
        
        WQ_splits = self.split_heads(WQ)
        WK_splits = self.split_heads(WK)
        WV_splits = self.split_heads(WV)
        
        out, attention_weights = self.scaled_dot_product_attention(
            WQ_splits, WK_splits, WV_splits, mask
        )
        
        out = self.combine_heads(out)
        out = self.linear(out)
        
        return out, attention_weights

In [21]:
class PoswiseFeedForwardNet(tf.keras.layers.Layer):
    def __init__(self, d_model, d_ff):
        super(PoswiseFeedForwardNet, self).__init__()
        self.w_1 = tf.keras.layers.Dense(d_ff, activation='relu')
        self.w_2 = tf.keras.layers.Dense(d_model)
        
    def call(self, x):
        out = self.w_1(x)
        out = self.w_2(out)
        return out

In [22]:
class EncoderLayer(tf.keras.layers.Layer):
    def __init__(self, d_model, n_heads, d_ff, dropout):
        super(EncoderLayer, self).__init__()
        self.enc_self_attn = MultiHeadAttention(d_model, n_heads)
        self.ffn = PoswiseFeedForwardNet(d_model, d_ff)
        
        self.norm_1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.norm_2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        
        self.do = tf.keras.layers.Dropout(dropout)
        
    def call(self, x, mask):
        #- Multi-Head Attention
        residual = x
        out = self.norm_1(x)
        out, enc_attn = self.enc_self_attn(out, out, out, mask)
        out = self.do(out)
        out += residual
        
        #- Position-Wise Feed Forward Network
        residual = out
        out = self.norm_2(out)
        out = self.ffn(out)
        out = self.do(out)
        out += residual
        
        return out, enc_attn

In [23]:
class DecoderLayer(tf.keras.layers.Layer):
    def __init__(self, d_model, num_heads, d_ff, dropout):
        super(DecoderLayer, self).__init__()

        self.dec_self_attn = MultiHeadAttention(d_model, num_heads)
        self.enc_dec_attn = MultiHeadAttention(d_model, num_heads)

        self.ffn = PoswiseFeedForwardNet(d_model, d_ff)

        self.norm_1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.norm_2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.norm_3 = tf.keras.layers.LayerNormalization(epsilon=1e-6)

        self.do = tf.keras.layers.Dropout(dropout)
        
    def call(self, x, enc_out, causality_mask, padding_mask):
        #- Masked Multi-Head Attention
        residual = x
        out = self.norm_1(x)
        out, dec_attn = self.dec_self_attn(out, out, out, padding_mask)
        out = self.do(out)
        out += residual
        
        #- Multi-Head Attention
        residual = out
        out = self.norm_2(out)
        out, dec_enc_attn = self.enc_dec_attn(out, enc_out, enc_out, causality_mask)
        out = self.do(out)
        out += residual

        #- Position-Wise Feed Forward Network
        residual = out
        out = self.norm_3(out)
        out = self.ffn(out)
        out = self.do(out)
        out += residual

        return out, dec_attn, dec_enc_attn

In [24]:
class Encoder(tf.keras.Model):
    def __init__(self, n_layers, d_model, n_heads, d_ff, dropout):
        super(Encoder, self).__init__()
        self.n_layers = n_layers
        self.enc_layers = [EncoderLayer(d_model, n_heads, d_ff, dropout) for _ in range(n_layers)]
        self.do = tf.keras.layers.Dropout(dropout)
        
    def call(self, x, mask):
        out = x
        enc_attns = list()
        for i in range(self.n_layers):
            out, enc_attn = self.enc_layers[i](out, mask)
            enc_attns.append(enc_attn)
            
        return out, enc_attns

In [25]:
class Decoder(tf.keras.Model):
    def __init__(self, n_layers, d_model, n_heads, d_ff, dropout):
        super(Decoder, self).__init__()
        self.n_layers = n_layers
        self.dec_layers = [DecoderLayer(d_model, n_heads, d_ff, dropout) for _ in range(n_layers)]
        
    def call(self, x, enc_out, causality_mask, padding_mask):
        out = x
        dec_attns = list()
        dec_enc_attns = list()
        for i in range(self.n_layers):
            out, dec_attn, dec_enc_attn = self.dec_layers[i](out, enc_out, causality_mask, padding_mask)
            dec_attns.append(dec_attn)
            dec_enc_attns.append(dec_enc_attn)
        
        return out, dec_attns, dec_enc_attns

In [26]:
class Transformer(tf.keras.Model):
    def __init__(self, n_layers, d_model, n_heads, d_ff, src_vocab_size, tgt_vocab_size,
                 pos_len, dropout=0.2, shared=True):
        super(Transformer, self).__init__()
        self.d_model = tf.cast(d_model, tf.float32)
        
        #- 1. Embedding Layer 정의
        self.enc_emb = tf.keras.layers.Embedding(src_vocab_size, d_model)
        self.dec_emb = tf.keras.layers.Embedding(tgt_vocab_size, d_model)
        
        #- 2. Positional Encoding 정의
        self.pos_encoding = positional_encoding(pos_len, d_model)
        #- 6. Dropout 정의
        self.do = tf.keras.layers.Dropout(dropout)
        
        #- 3. Encoder / Decoder 정의
        self.encoder = Encoder(n_layers, d_model, n_heads, d_ff, dropout)
        self.decoder = Decoder(n_layers, d_model, n_heads, d_ff, dropout)
        
        #- 4. Output Linear 정의
        self.fc = tf.keras.layers.Dense(tgt_vocab_size)
        
        #- 5. Shared Weights
        self.shared = shared
        
        if shared:
            self.fc.set_weights(tf.transpose(self.dec_emb.weights))
        
        
    def embedding(self, emb, x):
        """
        입력된 정수 배열을 Embedding + Pos Encoding
        + Shared일 경우 Scaling 작업 포함

        x: [ batch x length ]
        return: [ batch x length x emb ]
        """
        seq_len = x.shape[1]
        out = emb(x)
        
        if self.shared:
            out *= tf.math.sqrt(self.d_model)
        
        out += self.pos_encoding[np.newaxis, ...][:, :seq_len, :]
        out = self.do(out)
        
        return out
    
    def call(self, enc_in, dec_in, enc_mask, causality_mask, dec_mask):
        #- Step 1: Embedding(enc_in, dec_in) -> enc_in, dec_in
        enc_in = self.embedding(self.enc_emb, enc_in)
        dec_in = self.embedding(self.dec_emb, dec_in)
        
        #- Step 2: Encoder(enc_in, enc_mask) -> enc_out, enc_attns
        enc_out, enc_attns = self.encoder(enc_in, enc_mask)

        #- Step 3: Decoder(dec_in, enc_out, mask) -> dec_out, dec_attns, dec_enc_attns
        dec_out, dec_attns, dec_enc_attns = self.decoder(dec_in, enc_out, causality_mask, dec_mask)
        
        #- Step 4: Out Linear(dec_out) -> logits
        logits = self.fc(dec_out)
        
        return logits, enc_attns, dec_attns, dec_enc_attns

In [27]:
def generate_padding_mask(seq):
    seq = tf.cast(tf.math.equal(seq, 0), tf.float32)
    return seq[:, tf.newaxis, tf.newaxis, :]

def generate_causality_mask(src_len, tgt_len):
    mask = 1 - np.cumsum(np.eye(src_len, tgt_len), 0)
    return tf.cast(mask, tf.float32)

def generate_masks(src, tgt):
    enc_mask = generate_padding_mask(src)
    dec_mask = generate_padding_mask(tgt)

    dec_enc_causality_mask = generate_causality_mask(tgt.shape[1], src.shape[1])
    dec_enc_mask = tf.maximum(enc_mask, dec_enc_causality_mask)

    dec_causality_mask = generate_causality_mask(tgt.shape[1], tgt.shape[1])
    dec_mask = tf.maximum(dec_mask, dec_causality_mask)

    return enc_mask, dec_enc_mask, dec_mask

# 4. 훈련하기
앞서 필요한 것들을 모두 정의했기 때문에 우리는 훈련만 하면 됩니다! 아래 과정을 차근차근 따라가며 모델을 훈련하고, 예문에 대한 멋진 번역을 제출하세요!

1. 2 Layer를 가지는 Transformer를 선언하세요.(하이퍼파라미터는 자유롭게 조절합니다.)

2. 논문에서 사용한 것과 동일한 Learning Rate Scheduler를 선언하고, 이를 포함하는 Adam Optimizer를 선언하세요. (Optimizer의 파라미터 역시 논문과 동일하게 설정합니다.)

3. Loss 함수를 정의하세요.                                                         Sequence-to-sequence 모델에서 사용했던 Loss와 유사하되, Masking 되지 않은 입력의 개수로 Scaling하는 과정을 추가합니다. (트랜스포머가 모든 입력에 대한 Loss를 한 번에 구하기 때문입니다.)

4. train_step 함수를 정의하세요.                                                      입력 데이터에 알맞은 Mask를 생성하고, 이를 모델에 전달하여 연산에서 사용할 수 있게 합니다.

5. 학습을 진행합니다.                                                                매 Epoch 마다 제시된 예문에 대한 번역을 생성하고, 멋진 번역이 생성되면 그때의 하이퍼파라미터와 생성된 번역을 제출하세요!

**예문**
```
1. 오바마는 대통령이다.
2. 시민들은 도시 속에 산다.
3. 커피는 필요 없다.
4. 일곱 명의 사망자가 발생했다.
```

**결과**
```
결과(output)
Translations
> 1. obama is the president elect .
> 2. they are in the city .
> 3. they don t need to be a lot of drink .
> 4. seven other people have been killed in the attacks .

Hyperparameters
> n_layers: 2
> d_model: 512
> n_heads: 8
> d_ff: 2048
> dropout: 0.3

Training Parameters
> Warmup Steps: 4000
> Batch Size: 64
> Epoch At: 5
```

translate() 함수의 plot_attention 변수를 True 로 주면 번역 결과에 대한 Attention Map을 시각화 해볼 수 있습니다.

In [28]:
transformer = Transformer(
    n_layers=2,
    d_model=512,
    n_heads=8,
    d_ff = 2048,
    src_vocab_size=SRC_VOCAB_SIZE,
    tgt_vocab_size=TGT_VOCAB_SIZE,
    pos_len=200,
    dropout=0.2,
    shared=True
)

In [29]:
class LearningRateScheduler(tf.keras.optimizers.schedules.LearningRateSchedule):
    def __init__(self, d_model, warmup_steps=4000):
        super(LearningRateScheduler, self).__init__()
        self.d_model = d_model
        self.warmup_steps = warmup_steps
    
    def __call__(self, step):
        arg1 = step ** -0.5
        arg2 = step * (self.warmup_steps ** -1.5)
        
        return (self.d_model ** -0.5) * tf.math.minimum(arg1, arg2)

In [30]:
learning_rate = LearningRateScheduler(512)
optimizer = tf.keras.optimizers.Adam(learning_rate, beta_1=0.9, beta_2=0.98, epsilon=1e-9)

In [31]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

def loss_function(real, pred):
    mask = tf.math.logical_not(tf.math.equal(real, 0))
    loss_ = loss_object(real, pred)

    # Masking 되지 않은 입력의 개수로 Scaling하는 과정
    mask = tf.cast(mask, dtype=loss_.dtype)
    loss_ *= mask

    return tf.reduce_sum(loss_)/tf.reduce_sum(mask)

In [32]:
# Train Step 함수 정의

@tf.function()
def train_step(src, tgt, model, optimizer):
    gold = tgt[:, 1:]
        
    enc_mask, dec_enc_mask, dec_mask = generate_masks(src, tgt)

    # 계산된 loss에 tf.GradientTape()를 적용해 학습을 진행합니다.
    with tf.GradientTape() as tape:
        predictions, enc_attns, dec_attns, dec_enc_attns = \
        model(src, tgt, enc_mask, dec_enc_mask, dec_mask)
        loss = loss_function(gold, predictions[:, :-1])

    # 최종적으로 optimizer.apply_gradients()가 사용됩니다. 
    gradients = tape.gradient(loss, model.trainable_variables)    
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    
    return loss, enc_attns, dec_attns, dec_enc_attns

In [33]:
# Attention 시각화 함수

def visualize_attention(src, tgt, enc_attns, dec_attns, dec_enc_attns):
    def draw(data, ax, x="auto", y="auto"):
        import seaborn
        seaborn.heatmap(data, 
                        square=True,
                        vmin=0.0, vmax=1.0, 
                        cbar=False, ax=ax,
                        xticklabels=x,
                        yticklabels=y)
        
    for layer in range(0, 2, 1):
        fig, axs = plt.subplots(1, 4, figsize=(20, 10))
        print("Encoder Layer", layer + 1)
        for h in range(4):
            draw(enc_attns[layer][0, h, :len(src), :len(src)], axs[h], src, src)
        plt.show()
        
    for layer in range(0, 2, 1):
        fig, axs = plt.subplots(1, 4, figsize=(20, 10))
        print("Decoder Self Layer", layer+1)
        for h in range(4):
            draw(dec_attns[layer][0, h, :len(tgt), :len(tgt)], axs[h], tgt, tgt)
        plt.show()

        print("Decoder Src Layer", layer+1)
        fig, axs = plt.subplots(1, 4, figsize=(20, 10))
        for h in range(4):
            draw(dec_enc_attns[layer][0, h, :len(tgt), :len(src)], axs[h], src, tgt)
        plt.show()

In [34]:
# 번역 생성 함수

def evaluate(sentence, model, src_tokenizer, tgt_tokenizer):
    sentence = preprocess_sentence(sentence)

    pieces = src_tokenizer.encode_as_pieces(sentence)
    tokens = src_tokenizer.encode_as_ids(sentence)

    _input = tf.keras.preprocessing.sequence.pad_sequences([tokens],
                                                           maxlen=enc_train.shape[-1],
                                                           padding='post')
    
    ids = []
    output = tf.expand_dims([tgt_tokenizer.bos_id()], 0)
    for i in range(dec_train.shape[-1]):
        enc_padding_mask, combined_mask, dec_padding_mask = \
        generate_masks(_input, output)

        predictions, enc_attns, dec_attns, dec_enc_attns =\
        model(_input, 
              output,
              enc_padding_mask,
              combined_mask,
              dec_padding_mask)

        predicted_id = \
        tf.argmax(tf.math.softmax(predictions, axis=-1)[0, -1]).numpy().item()

        if tgt_tokenizer.eos_id() == predicted_id:
            result = tgt_tokenizer.decode_ids(ids)
            return pieces, result, enc_attns, dec_attns, dec_enc_attns

        ids.append(predicted_id)
        output = tf.concat([output, tf.expand_dims([predicted_id], 0)], axis=-1)

    result = tgt_tokenizer.decode_ids(ids)

    return pieces, result, enc_attns, dec_attns, dec_enc_attns

In [35]:
# 번역 생성 및 Attention 시각화 결합

def translate(sentence, model, src_tokenizer, tgt_tokenizer, plot_attention=False):
    pieces, result, enc_attns, dec_attns, dec_enc_attns = \
    evaluate(sentence, model, src_tokenizer, tgt_tokenizer)
    
    print('Input: %s' % (sentence))
    print('Predicted translation: {}'.format(result))

    if plot_attention:
        visualize_attention(pieces, result.split(), enc_attns, dec_attns, dec_enc_attns)

In [38]:
# 학습

from tqdm import tqdm_notebook 

BATCH_SIZE = 64
EPOCHS = 30

examples = [
            "오바마는 대통령이다.",
            "시민들은 도시 속에 산다.",
            "커피는 필요 없다.",
            "일곱 명의 사망자가 발생했다.",
            "넌 젖을수록 투명해지는 꽃",
            "우리 사이 흰 꽃잎이",
            "후회로 촉촉해져 가.",
            "투명하지만 사라지진 않아",
            "보이지 않을 땐",
            "아프지라도 않던데",
            "알고도 잡지 못하는 게 ",
            "찢어질 듯 죽을 듯 아프구나",
            "눈물에 흠뻑 젖어버리니",
            "뻔한 내 잘못은 이젠 안 보여",
            "아 바람에 흩날리는구나",
            "아 이슬에 젖어가는구나",
            "코앞의 꽃잎아",
            "넌 날 속인 진한",
            "향기를 품고",
            "그댈 찾을 수 없도록",
            "영원의 방에 날 가두고",
            "간사하게 새하얗게 웃고 있구나",
            "아 바람에 흩날리구나",
            "아 이슬에 젖어가는구나",
            "시간이 지나 흰 꽃잎들도",
            "투명해진 기억 없이",
            "시들어 가겠지",
            "아 슬픔에 흩날리는구나",
            "아 눈물에 젖어가는구나",
            "시간이 지나"         
]

for epoch in range(EPOCHS):
    total_loss = 0
    
    idx_list = list(range(0, enc_train.shape[0], BATCH_SIZE))
    random.shuffle(idx_list)
    t = tqdm_notebook(idx_list)

    for (batch, idx) in enumerate(t):
        batch_loss, enc_attns, dec_attns, dec_enc_attns = \
        train_step(enc_train[idx:idx+BATCH_SIZE],
                    dec_train[idx:idx+BATCH_SIZE],
                    transformer,
                    optimizer)

        total_loss += batch_loss
        
        t.set_description_str('Epoch %2d' % (epoch + 1))
        t.set_postfix_str('Loss %.4f' % (total_loss.numpy() / (batch + 1)))

    for example in examples:
        translate(example, transformer, ko_tokenizer, en_tokenizer)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:46: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


  0%|          | 0/1127 [00:00<?, ?it/s]

Input: 오바마는 대통령이다.
Predicted translation: obama is the first president .
Input: 시민들은 도시 속에 산다.
Predicted translation: we re eager to see the city in the city .
Input: 커피는 필요 없다.
Predicted translation: didn t need for up to geneva next
Input: 일곱 명의 사망자가 발생했다.
Predicted translation: seven civilian people were killed , including seven deaths .
Input: 넌 젖을수록 투명해지는 꽃
Predicted translation: the more babies , useless tend to be unable to be unable to be unable by the sounds of chill with the air .
Input: 우리 사이 흰 꽃잎이
Predicted translation: a white pizza white begins the white sea white genitalia , white phones with white beach , as a white regular treaty white treating white shuttle .
Input: 후회로 촉촉해져 가.
Predicted translation: that is dead . . .
Input: 투명하지만 사라지진 않아
Predicted translation: to not , un resolve beijing .
Input: 보이지 않을 땐
Predicted translation: b saw up presents http article .
Input: 아프지라도 않던데
Predicted translation: 
Input: 알고도 잡지 못하는 게 
Predicted translation: and progressing magazi

  0%|          | 0/1127 [00:00<?, ?it/s]

Input: 오바마는 대통령이다.
Predicted translation: obama is the first president to speak today .
Input: 시민들은 도시 속에 산다.
Predicted translation: the wounded city will suffer a owners of acid .
Input: 커피는 필요 없다.
Predicted translation: didn t need for children
Input: 일곱 명의 사망자가 발생했다.
Predicted translation: seven deaths seven others were killed and seven were reported monday .
Input: 넌 젖을수록 투명해지는 꽃
Predicted translation: we re watching from work , the effort gets whipped make a transparent and sound profit on more transparent , cash , the sounds we want to make sure that speed is more captives can be allowed out .
Input: 우리 사이 흰 꽃잎이
Predicted translation: white is a white guinea among white white things we re going through the white waters .
Input: 후회로 촉촉해져 가.
Predicted translation: . gain nearly youths .
Input: 투명하지만 사라지진 않아
Predicted translation: prompted un mspecified ran unspecified damages tightly and un october off .
Input: 보이지 않을 땐
Predicted translation: b snow faces http article .
Input: 아프지라

...

Input: 오바마는 대통령이다.
Predicted translation: the president s second presidential aide , obama is inside .
Input: 시민들은 도시 속에 산다.
Predicted translation: the city was the th city of land in welive cities .
Input: 커피는 필요 없다.
Predicted translation: didn t need a chance to take a step down at the base .
Input: 일곱 명의 사망자가 발생했다.
Predicted translation: the seven death toll in seven story .
Input: 넌 젖을수록 투명해지는 꽃
Predicted translation: the commander gets a negative image foremost t allows unless , the teenagers watch , the level can count on o cheering with possible cast a illegal drug containing .
Input: 우리 사이 흰 꽃잎이
Predicted translation: a white restaurants among white white white restaurants kicked off white carriers .
Input: 후회로 촉촉해져 가.
Predicted translation: by monday , the group is conducting a tsunami .
Input: 투명하지만 사라지진 않아
Predicted translation: openly unupper nuclear power ms unprecedented in consumer commercial peru , not enough to transparently .
Input: 보이지 않을 땐
Predicted translation: htt

  0%|          | 0/1127 [00:00<?, ?it/s]

Input: 오바마는 대통령이다.
Predicted translation: obama is a first taken kind .
Input: 시민들은 도시 속에 산다.
Predicted translation: we re on the city , one of the city s big town hall ministry said .
Input: 커피는 필요 없다.
Predicted translation: didn t need a
Input: 일곱 명의 사망자가 발생했다.
Predicted translation: seven deaths were rescued .
Input: 넌 젖을수록 투명해지는 꽃
Predicted translation: the more babies , are continues to be distributed with flowers , the workers where unable , leading make money , is more efficient between the rival reality .
Input: 우리 사이 흰 꽃잎이
Predicted translation: a white high white restaurants kick with white white woman s white st whitevery clearly and it is clear we re s white potato .
Input: 후회로 촉촉해져 가.
Predicted translation: by monday , the group is dead .
Input: 투명하지만 사라지진 않아
Predicted translation: not to get commercials un million ran away from the carri .
Input: 보이지 않을 땐
Predicted translation: less http article . joins . com article cnn e article . asp ?
Input: 아프지라도 않던데
Predicted transl

  0%|          | 0/1127 [00:00<?, ?it/s]

Input: 오바마는 대통령이다.
Predicted translation: obama is a first term .
Input: 시민들은 도시 속에 산다.
Predicted translation: the city was one of the city s movie .
Input: 커피는 필요 없다.
Predicted translation: didn t need a
Input: 일곱 명의 사망자가 발생했다.
Predicted translation: seven deaths seven were totally destroyed , .
Input: 넌 젖을수록 투명해지는 꽃
Predicted translation: the super bowl runs meansly do the show make flowers that can more efficiently effectively and make flowers more efficient , available , and for fire week cast , we do not want contribute to a quickly transparent long and afford .
Input: 우리 사이 흰 꽃잎이
Predicted translation: a white restaurants kick white it s a white st . victory white used white restaurants with white st .y white st slogans .
Input: 후회로 촉촉해져 가.
Predicted translation: by monday , the greek guys are conducting a l . local county , which is conducted by local time objects in the end of the l .
Input: 투명하지만 사라지진 않아
Predicted translation: not to comply with a failed commercial stage .
Inp

# 회고
- 예문을 학습시켜보았고, 이외에도 노래가사를 넣어서 학습을 시켰으나 그렇게 만족스럽진 않았다. 

- 학습을 많이 시킨다고 내가 원하는 방향으로는 번역이 잘 되지는 않았다               ex) [흰 꽃잎] 이라는 단어가 나왔는데 꽃잎은 번역되지 않고 white 만 번역이 된걸 알수 있었다. 

- 여러번 학습 시킨 결과 아직 많이 번역이 부족한거 같다라고 생각한다.
